<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#benchmarking-to_numpy" data-toc-modified-id="benchmarking-to_numpy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>benchmarking to_numpy</a></span><ul class="toc-item"><li><span><a href="#Convert-image-uri-to-blob" data-toc-modified-id="Convert-image-uri-to-blob-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Convert image uri to blob</a></span></li><li><span><a href="#Converting-image-to-numpy-array-without-copies" data-toc-modified-id="Converting-image-to-numpy-array-without-copies-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Converting image to numpy array without copies</a></span></li></ul></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext cython

In [2]:
import time
import jina
from jina import Document, DocumentArray
import random
import glob
from PIL import Image
import numpy as np


In [3]:
#files = glob.glob('/Volumes/corsair2tv/data/imaterialist-fashion-2019-FGVC6/test/*.jpg')
files = glob.glob('./team_fotos/*.jpg')

In [4]:
len(files)
source = files[0]
files[0:2]

['./team_fotos/image_from_ios 2.jpg', './team_fotos/image_from_ios-2.jpg']

In [5]:
from jina.types.document.converters import to_image_blob

In [7]:
aux = to_image_blob(files[0])

In [26]:
%timeit aux = to_image_blob(files[0])

269 ms ± 6.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
def image_generator(files):
    da = DocumentArray()
    for idx, filename in enumerate(files):
        doc = Document(uri=filename)
        doc.convert_image_uri_to_blob()
        da.append(doc)

In [25]:
%timeit aux = image_generator(files[0:1])

453 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
def to_image_blob_new(source, color_axis: int = -1) -> 'np.ndarray':
    """
    Convert an image buffer to blob

    :param source: image bytes buffer
    :param color_axis: the axis id of the color channel, ``-1`` indicates the color channel info at the last axis
    :return: image blob
    """
    from PIL import Image

    raw_img = Image.open(source)
    img = np.asarray(raw_img, dtype='float32')
    if color_axis != -1:
        img = np.moveaxis(img, -1, color_axis)
    return img

In [ ]:
%timeit aux = to_image_blob_new(files[0])

In [ ]:
def to_image_blob_new2(source, color_axis: int = -1) -> 'np.ndarray':
    """
    Convert an image buffer to blob

    :param source: image bytes buffer
    :param color_axis: the axis id of the color channel, ``-1`` indicates the color channel info at the last axis
    :return: image blob
    """
    from PIL import Image

    raw_img = Image.open(source)
    shape, typestr = Image._conv_type_shape(raw_img)
    img = np.empty(shape, dtype='float32')
    img = np.asarray(raw_img, dtype='float32')
    if color_axis != -1:
        img = np.moveaxis(img, -1, color_axis)
    return img

In [ ]:
%timeit aux = to_image_blob_new2(files[0])

In [ ]:
raw_img = Image.open(source)
%timeit raw_img = Image.open(source)

shape, typestr = Image._conv_type_shape(raw_img)
%timeit shape, typestr = Image._conv_type_shape(raw_img)

img = np.empty(shape, dtype='float32')
%timeit img = np.empty(shape, dtype='float32')

%timeit img = np.asarray(raw_img, dtype='float32')

In [ ]:
%timeit aux = raw_img.tobytes()

In [ ]:
def to_numpy(raw_img):
    raw_img.load()
    e = Image._getencoder(raw_img.mode, 'raw', raw_img.mode)
    e.setimage(raw_img.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(raw_img)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

def to_numpy(raw_img):
    cache = 2**16
    
    raw_img.load()
    e = Image._getencoder(raw_img.mode, 'raw', raw_img.mode)
    e.setimage(raw_img.im)

    shape, typestr = Image._conv_type_shape(raw_img)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    bufsize, stop_flag, offset = cache, 0, 0 
    while stop_flag == 0:
        l, stop_flag, im_chunk = e.encode(bufsize)
        mem[offset:offset + len(im_chunk)] = im_chunk
        offset += len(im_chunk)

    return data

def to_image_blob_new3(source, color_axis: int = -1) -> 'np.ndarray':
    """
    Convert an image buffer to blob

    :param source: image bytes buffer
    :param color_axis: the axis id of the color channel, ``-1`` indicates the color channel info at the last axis
    :return: image blob
    """
    from PIL import Image

    raw_img = Image.open(source)
    img = raw_img_to_numpy(raw_img)
    if color_axis != -1:
        img = np.moveaxis(img, -1, color_axis)
    return img


In [ ]:
raw_img = Image.open(source)
type(raw_img)

In [ ]:
%timeit aux3 = to_image_blob_new3(files[0])

In [ ]:
%timeit aux2 = to_image_blob_new2(files[0])

In [ ]:
%timeit aux1 = to_image_blob_new(files[0])

In [ ]:
%timeit aux0 = to_image_blob(files[0])

In [ ]:
aux = Image.open(source)

In [ ]:
%timeit aux.size

In [ ]:
%timeit aux.tobytes()

In [ ]:
def image_generator2(files):
    da = DocumentArray()
    for idx, filename in enumerate(files):
        doc = Document(uri=filename)
        doc.convert_image_uri_to_blob()
        da.append(doc)

In [ ]:
%time aux = image_generator(files)

In [ ]:
def image_generator2(files):
    da = DocumentArray()
    for idx, filename in enumerate(files):
        doc = Document(uri=filename)
        doc.convert_image_uri_to_blob()
        da.append(doc)

In [ ]:
##

## Cythonizing code

In [ ]:
raw_img.load()
e = Image._getencoder(raw_img.mode, 'raw', raw_img.mode)
e.setimage(raw_img.im)

In [ ]:
def to_numpy(raw_img):
    cache = 2**16
    
    raw_img.load()
    e = Image._getencoder(raw_img.mode, 'raw', raw_img.mode)
    e.setimage(raw_img.im)

    shape, typestr = Image._conv_type_shape(raw_img)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    bufsize, stop_flag, offset = cache, 0, 0 
    while stop_flag == 0:
        l, stop_flag, im_chunk = e.encode(bufsize)
        mem[offset:offset + len(im_chunk)] = im_chunk
        offset += len(im_chunk)

    return data

In [ ]:
%timeit img = to_numpy(raw_img)

In [ ]:
raw_img.size, img.shape

If we load all the data into memory it does not fit in L1 cache and it becomes less performant

In [ ]:

def to_numpy2(raw_img):
    
    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(raw_img)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    mem[0:] = raw_img.tobytes()
    
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data 

In [ ]:
raw_img = Image.open(source)

In [ ]:
# note that this is very slow due to 
# memory having to go to RAM vs beein in cache
%timeit mem = raw_img.tobytes()

In [ ]:
raw_img = Image.open(source)
%timeit img = to_numpy2(raw_img)

We can cythonize the loop to achieve better performance

In [ ]:
%%cython -a
cimport numpy as cnp
import numpy as np
cimport cython
from PIL import Image 

@cython.boundscheck(False)
@cython.wraparound(False)

cdef fill_in_array(array, mem, n_x, n_y, n_c):
    for 

def to_numpy3(raw_img):

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(raw_img)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    bufsize, s, offset = 65536, 0, 0
    #mem[0:] = raw_img.tobytes()
    n_x, n_y, n_c = shape    
    fill_in_array(data, mem, n_x, n_y, n_c)
    
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data 
        

In [ ]:
%time img = to_numpy3(raw_img)

In [ ]:
#Same as
#data.shape[0] * data.shape[1] * data.shape[2]  
len(mem)

### benchmarking to_numpy

In [ ]:
64000/2**14

In [ ]:
def to_numpy(im_path):
    #im = Image.open(im_path)
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data

In [ ]:
for i in range(6, 0, -1):
    i = 128 * 2 ** i
    print(f'\n\nSize: {i}x{i}   \t{i*i // 1024} KPx')
    im = Image.new('RGB', (i, i))
    print('\tnumpy.array()')
    %timeit n = np.array(im)
    print('\tnumpy.asarray()')
    %timeit n = np.asarray(im)
    print('\tto_numpy()')
    %timeit n = to_numpy(im)
    im = None

In [ ]:
data,

#### Convert image uri to blob


In the following lines we can see that the method `convert_image_uri_to_blob` takes a considerable time

Note tha the function
```python
    def convert_image_uri_to_blob(
        self, color_axis: int = -1, uri_prefix: Optional[str] = None
    ):
        self.blob = to_image_blob(
            (uri_prefix + self.uri) if uri_prefix else self.uri, color_axis
        )
 ```
   
Is basically calling `to_image_blob`
   
```python
def to_image_blob(source, color_axis: int = -1) -> 'np.ndarray':
    from PIL import Image

    raw_img = Image.open(source).convert('RGB')
    img = np.array(raw_img).astype('float32')
    if color_axis != -1:
        img = np.moveaxis(img, -1, color_axis)
    return img

```

In [ ]:
doc = Document(uri=files[0])

In [ ]:
%time doc.convert_image_uri_to_blob()

In [ ]:
doc = Document(uri=files[0])
doc.convert_image_uri_to_blob()
doc.blob.shape, doc.blob.dtype

In [ ]:
from PIL import Image
source=files[0]
raw_img = Image.open(source).convert('RGB')

In [ ]:
%timeit raw_img = Image.open(source)

In [ ]:
color_axis = -1
raw_img = Image.open(source).convert('RGB')
img = np.array(raw_img).astype('float32')
if color_axis != -1:
    img = np.moveaxis(img, -1, color_axis)

In [ ]:
img.shape

In [ ]:
color_axis = -1
raw_img2 = Image.open(source)
img2 = np.array(raw_img2).astype('float32')
if color_axis != -1:
    img2 = np.moveaxis(img, -1, color_axis)

In [ ]:
img2.shape, img2.dtype

In [ ]:
type(raw_img2), type(raw_img2.convert('RGB'))

In [ ]:
%timeit aux = Image.open(source)

In [ ]:
%timeit aux = Image.open(source)
%timeit img = np.asarray(raw_img, dtype='float32')

In [ ]:
%timeit img = np.asarray(raw_img).astype('float32')

In [ ]:
%timeit img = np.array(raw_img).astype('float32')

#### Converting image to numpy array without copies

In [ ]:
import numpy 


In [ ]:
img = Image.open(source)
shape, typestr = Image._conv_type_shape(img)
data = numpy.empty(shape, dtype=numpy.dtype(typestr))
mem = data.data.cast('B', (data.data.nbytes,))

In [ ]:
#data[0],
np.array(mem).reshape(shape)[0]

In [ ]:
%timeit img = np.array(Image.open(source))

In [ ]:
%timeit data = numpy.empty(shape, dtype=numpy.dtype(typestr))

In [ ]:
shape, typestr = Image._conv_type_shape(im)
data = numpy.empty(shape, dtype=numpy.dtype(typestr))
mem = data.data.cast('B', (data.data.nbytes,))

bufsize, s, offset = 65536, 0, 0

In [ ]:
%timeit e = Image._getencoder(img.mode, 'raw', img.mode)

In [ ]:
e = Image._getencoder(img.mode, 'raw', img.mode)
e.setimage(im)

In [ ]:
np.frombuffer(img)

In [ ]:

e.setimage(img.im)
shape, typestr = Image._conv_type_shape(img)
data = numpy.empty(shape, dtype=numpy.dtype(typestr))
mem = data.data.cast('B', (data.data.nbytes,))

bufsize, s, offset = 65536, 0, 0
while not s:
    l, s, d = e.encode(bufsize)
    mem[offset:offset + len(d)] = d
    offset += len(d)

In [ ]:
result = np.asarray(mem).reshape(shape)

In [ ]:
def to_mem(source):
    img = Image.open(source)

    e = Image._getencoder(im.mode, "raw", img.mode)
    e.setimage(img.im)

    
    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = numpy.empty(shape, dtype=numpy.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))


    bufsize, offset, s = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError(f"encoder error {s} in tobytes")
    return mem

In [ ]:
to_mem(files[0])

In [ ]:
def to_image_blob_new(source, color_axis: int = -1) -> 'np.ndarray':

    from PIL import Image

    img = Image.open(source)
    shape, typestr = Image._conv_type_shape(img)
    data = numpy.empty(shape, dtype=numpy.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))
    
    if color_axis != -1:
        img = np.moveaxis(img, -1, color_axis)
    return mem


In [ ]:
aux = to_image_blob_new(files[0])

In [ ]:
mem

In [ ]:
def image_generator2(files):
    da = DocumentArray()
    for idx, filename in enumerate(files):
        doc = Document(uri=filename)
        doc.convert_image_uri_to_blob()
        da.append(doc)

In [ ]:
%time aux = image_generator(files[0:50])

In [ ]:
def to_numpy(im_path):
    im = Image.open(im_path)
    im.load()
    # unpack data
    e = Image._getencoder(im.mode, 'raw', im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = numpy.empty(shape, dtype=numpy.dtype(typestr))
    mem = data.data.cast('B', (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)
        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data


In [ ]:
%timeit to_numpy(files[0])

In [ ]:
im = Image.open(files[0])

%timeit e = Image._getencoder(im.mode, 'raw', im.mode)

In [ ]:
e = Image._getencoder(im.mode, 'raw', im.mode)

In [ ]:
shape, typestr = Image._conv_type_shape(im)

In [ ]:
%timeit data = numpy.empty(shape, dtype=numpy.dtype(typestr))